<a href="https://colab.research.google.com/github/sandipanbasu/aiml-capstone/blob/master/MRC_CNN_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [20]:
import tensorflow as tf
import pickle

import pandas as pd
import numpy as np

import os
from os import listdir


In [21]:
tf.__version__

'2.2.0'

In [38]:
projectPathCnn = "/content/drive/My Drive/AIML-MRC-Capstone/datasets/CNN/TrainingDatasets/"
csvFileName = "cnnTrainingData.csv"
cnnOriginalTrainDfName = "cnnOriginalTrainDf.csv"

quesZipFile = "cnn.tgz"


### Connecting to colab drive

In [23]:
!pip install -U -q PyDrive

In [24]:

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [25]:
!ls

adc.json  drive  sample_data


### CNN Question Processing

In [27]:
#!tar -xvf "/content/drive/My Drive/AIML-MRC-Capstone/datasets/CNN/TrainingDatasets/cnn.tgz"

Streaming output truncated to the last 5000 lines.
./cnn/questions/validation/4bc65727d5970944eabdac452bd93be490912b46.question
./cnn/questions/validation/8b4d9fdc0cfc2273404cae26d0e07875cd7d5121.question
./cnn/questions/validation/a8942176b2e1448b3f3a8af48611188d9ab4862d.question
./cnn/questions/validation/102c0d4d07a55e2b0652cebdb460d5ae83c590b0.question
./cnn/questions/validation/b0a8c1ab92a5a934d2f907622677fb8848ddcaa6.question
./cnn/questions/validation/ad90e1dbcef734584f64e3106b595828525f3645.question
./cnn/questions/validation/45cfabf1bcaa92950784540dc01b0526a2d8f01a.question
./cnn/questions/validation/6745b34cb7071db84ce2bb85f2919495dcbdeaf9.question
./cnn/questions/validation/a1ab146a8308cea7b27599b2a71b956f02ecf234.question
./cnn/questions/validation/1e1c3c96e53bcf41935ba2e0a56ab4bb720d9da4.question
./cnn/questions/validation/25001d356dba8ce66f53d50fb1ed53b60bb69d39.question
./cnn/questions/validation/e07456f20263f07eedd45de114db26b69f1e1349.question
./cnn/questions/validatio

In [28]:
!ls

adc.json  cnn  drive  sample_data


In [30]:
queslisttrain = os.listdir('/content/cnn/questions/training') # dir is your directory path
number_files_ques_train = len(queslisttrain)
number_files_ques_train

380298

In [31]:
def load_ques_doc(filename):
	# open the file as read only
	file = open(filename, encoding='utf-8')
	# read all text
	text = file.read()
	# close the file
	file.close()
	return text

# load all stories in a directory
def load_ques_content(directory):
	stories = []
	for name in listdir(directory):
		filename = directory + '/' + name
		# load document
		doc = load_ques_doc(filename)
		# split into story and highlights
		url, context, question, answer, entityMapping = split_content(doc)
		# store
		stories.append({'url':url, 'context':context, 'question':question, 'answer':answer, 'entityMapping':entityMapping})
	return stories


# split a document into news story and highlights  
def split_content(doc):
	# splitting the doc in sections
  content_list = doc.splitlines()
  lastitm = len(content_list)-1
  entityMapList = content_list[8 :lastitm]
	
	# split into story and highlights
  url = content_list[0]
  context = content_list[2]
  question = content_list[4]
  answer = content_list[6]
  entityMapping = entityMapList
  
  return url, context, question, answer, entityMapping

In [32]:
directory = '/content/cnn/questions/training/'
stories = load_ques_content(directory)

In [33]:
len (stories)

380298

In [34]:
cnnOriginalTrainDf = pd.DataFrame(stories)

In [35]:
cnnOriginalTrainDf.head(5)

,url,context,question,answer,entityMapping
0,http://web.archive.org/web/20130813202430id_/h...,( @entity0 ) -- this week 's devastating crash...,@placeholder : high - speed rail could revolut...,@entity175,"[@entity20:U.S., @entity80:France, @entity81:J..."
1,http://web.archive.org/web/20090327041133id_/h...,"( @entity0 ) -- more than 2,000 students in @e...",@placeholder students want to break 40 - mile ...,@entity2,"[@entity17:Auto Club Speedway, @entity2:Southe..."
2,http://web.archive.org/web/20100423183617id_/h...,"@entity1 , @entity0 ( @entity2 ) -- if treatin...",@placeholder program gives daily injections to...,@entity12,"[@entity2:CNN, @entity1:London, @entity0:Engla..."
3,http://web.archive.org/web/20140709072953id_/h...,"@entity0 , @entity1 ( @entity2 ) -- @entity3 h...","@placeholder called its actions "" justifiable ...",@entity3,"[@entity3:North Korea, @entity2:CNN, @entity1:..."
4,http://web.archive.org/web/20141221112546id_/h...,( @entity0 ) -- after nearly seven weeks on th...,new : prosecutors intend to seek the death pen...,@entity3,"[@entity112:Swiftwater, @entity111:Cory Batzel..."


In [39]:
#cnn_df.to_pickle("/content/drive/My Drive/cnn-dailymail/storiesdf.pkl")
cnnOriginalTrainDf.to_csv(projectPathCnn+cnnOriginalTrainDfName)

## Loading Data from CSV

In [40]:
cnnTrainDf = pd.read_csv(projectPathCnn+cnnOriginalTrainDfName)

In [41]:
def replace_ans_entity(ans,entlst):
  entstr = entlst[2:]
  entstr = entstr[:-1]
  entityList = entstr.split(',') #0th item <class 'str'> :  ['@entity21:Formula One', 1st item <class 'str'> :   '@entity24:Turkish Grand Prix', last item '@entity29:United States']
  entityListLen = len(entityList)

  for i in range(entityListLen):
    entitm = entityList[i].split(':')
    entity = entitm[0].replace("'", "")
    if len(entitm)>1:
      value = entitm[1].replace("'", "")
      #print(ans, " ",entity,": ",value)
      if ans.strip() == entity.strip():
        return(value.strip())
    else:
      return("None")





In [42]:
cnnTrainDf['ans_updated'] = cnnTrainDf.apply(lambda x: replace_ans_entity(x['answer'],x['entityMapping']),axis=1)

In [43]:
cnnTrainDf.head(10)

,Unnamed: 0,url,context,question,answer,entityMapping,ans_updated
0,0,http://web.archive.org/web/20130813202430id_/h...,( @entity0 ) -- this week 's devastating crash...,@placeholder : high - speed rail could revolut...,@entity175,"['@entity20:U.S.', '@entity80:France', '@entit...",Freemark
1,1,http://web.archive.org/web/20090327041133id_/h...,"( @entity0 ) -- more than 2,000 students in @e...",@placeholder students want to break 40 - mile ...,@entity2,"['@entity17:Auto Club Speedway', '@entity2:Sou...",Southern California
2,2,http://web.archive.org/web/20100423183617id_/h...,"@entity1 , @entity0 ( @entity2 ) -- if treatin...",@placeholder program gives daily injections to...,@entity12,"['@entity2:CNN', '@entity1:London', '@entity0:...",British
3,3,http://web.archive.org/web/20140709072953id_/h...,"@entity0 , @entity1 ( @entity2 ) -- @entity3 h...","@placeholder called its actions "" justifiable ...",@entity3,"['@entity3:North Korea', '@entity2:CNN', '@ent...",North Korea
4,4,http://web.archive.org/web/20141221112546id_/h...,( @entity0 ) -- after nearly seven weeks on th...,new : prosecutors intend to seek the death pen...,@entity3,"['@entity112:Swiftwater', '@entity111:Cory Bat...",Frein
5,5,http://web.archive.org/web/20140928231009id_/h...,( @entity0 ) -- as numerous news organizations...,the @placeholder government has denied that th...,@entity1,"['@entity31:Vladimir Putin', '@entity53:Wester...",Russian
6,6,http://web.archive.org/web/20120117001626id_/h...,( @entity0 ) -- dozens of people died as heavy...,13 people are killed in a mudslide in @placeho...,@entity11,"['@entity17:Laje', '@entity22:Minas Gerais', '...",Rio de Janeiro
7,7,http://web.archive.org/web/20090518165157id_/h...,( @entity0 ) -- at least 10 people were killed...,almost all of the victims are believed to be f...,@entity13,"['@entity0:CNN', '@entity13:Bahamas', '@entity...",Bahamas
8,8,http://web.archive.org/web/20130206190525id_/h...,( @entity0 ) -- @entity2 's fellow footballers...,@placeholder midfielder collapsed during match...,@entity4,"['@entity20:Manchester United', '@entity21:Twi...",Bolton
9,9,http://web.archive.org/web/20140915100819id_/h...,( @entity0 ) -- the @entity1 has placed 12 cou...,"@entity11 ranked second , while @entity67 and ...",@entity14,"['@entity20:Bangladesh', '@entity21:India', '@...",Colombia


In [ ]:
# Checking for duplicates
duplicateRowsDF = cnnTrainDf[cnnTrainDf.duplicated()]

In [44]:
print(duplicateRowsDF) #No Duplicte rows found

Empty DataFrame
Columns: [Unnamed: 0, url, context, question, answer, entityMapping, ans_updated]
Index: []


In [47]:
# Checking for null values
nulldf = cnnTrainDf.isnull() 

In [48]:
print(nulldf) # No missing values

        Unnamed: 0    url  context  ...  answer  entityMapping  ans_updated
0            False  False    False  ...   False          False        False
1            False  False    False  ...   False          False        False
2            False  False    False  ...   False          False        False
3            False  False    False  ...   False          False        False
4            False  False    False  ...   False          False        False
...            ...    ...      ...  ...     ...            ...          ...
380293       False  False    False  ...   False          False        False
380294       False  False    False  ...   False          False        False
380295       False  False    False  ...   False          False        False
380296       False  False    False  ...   False          False        False
380297       False  False    False  ...   False          False        False

[380298 rows x 7 columns]


In [49]:
cnnTrainDf.drop(['Unnamed: 0','url'],axis=1,inplace=True)

In [50]:
cnnTrainDf.head(5)

,context,question,answer,entityMapping,ans_updated
0,( @entity0 ) -- this week 's devastating crash...,@placeholder : high - speed rail could revolut...,@entity175,"['@entity20:U.S.', '@entity80:France', '@entit...",Freemark
1,"( @entity0 ) -- more than 2,000 students in @e...",@placeholder students want to break 40 - mile ...,@entity2,"['@entity17:Auto Club Speedway', '@entity2:Sou...",Southern California
2,"@entity1 , @entity0 ( @entity2 ) -- if treatin...",@placeholder program gives daily injections to...,@entity12,"['@entity2:CNN', '@entity1:London', '@entity0:...",British
3,"@entity0 , @entity1 ( @entity2 ) -- @entity3 h...","@placeholder called its actions "" justifiable ...",@entity3,"['@entity3:North Korea', '@entity2:CNN', '@ent...",North Korea
4,( @entity0 ) -- after nearly seven weeks on th...,new : prosecutors intend to seek the death pen...,@entity3,"['@entity112:Swiftwater', '@entity111:Cory Bat...",Frein


In [51]:
# saving the dataframe in csv

cnnTrainDf.to_csv(projectPathCnn+csvFileName)